In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

# from matplotlib import style
import seaborn as sns
%matplotlib inline

import wrangle

from env import geo_key

In [2]:
df = wrangle.get_curriculum_logs_data()

The df has 847330 rows and 10 columns.


In [3]:
df.head()

,cohortid,start_date,end_date,cohortname,program_id,date,time,path,user_id,ip,program_name,is_staff
0,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:47,/,11,97.105.19.61,web_dev,False
1,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:51,javascript-i,11,97.105.19.61,web_dev,False
2,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:15:02,javascript-i/functions,11,97.105.19.61,web_dev,False
3,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:29,appendix/further-reading/javascript/hoisting.html,11,97.105.19.61,web_dev,False
4,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:39,appendix,11,97.105.19.61,web_dev,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   cohortid      847330 non-null  int64         
 1   start_date    847330 non-null  datetime64[ns]
 2   end_date      847330 non-null  datetime64[ns]
 3   cohortname    847330 non-null  object        
 4   program_id    847330 non-null  int64         
 5   date          847330 non-null  datetime64[ns]
 6   time          847330 non-null  object        
 7   path          847329 non-null  object        
 8   user_id       847330 non-null  int64         
 9   ip            847330 non-null  object        
 10  program_name  847330 non-null  object        
 11  is_staff      847330 non-null  bool          
dtypes: bool(1), datetime64[ns](3), int64(3), object(5)
memory usage: 78.4+ MB


### Q3: Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [5]:
# Active access
df_active = df[(df.start_date <= df.date) & (df.date <= df.end_date)]

In [6]:
df_active.shape

(647496, 12)

In [7]:
# Curriculum access by counting the paths per user_id
df_active.groupby('user_id').path.count().describe()

count     731.000000
mean      885.766074
std       647.254908
min         1.000000
25%       391.500000
50%       795.000000
75%      1235.500000
max      4786.000000
Name: path, dtype: float64

In [8]:
# first quartile
q1 = np.quantile(df_active.groupby('user_id').path.count(), .25)

In [9]:
# find user id's in the first quartile of access count
inq1 = pd.DataFrame(df_active.groupby('user_id').path.count() < q1)
inq1.rename(columns = {'path':'in_q1'}, inplace=True)

In [10]:
# add column if the user_id is in the first quartile
df_active = pd.merge(df_active, inq1, how = 'right', on='user_id')

In [11]:
# dataframe of students who hardly accessed curriculum while active
df_swhacwa = df_active[df_active.in_q1 == True]

In [12]:
# Number of students who hardly accessed curriculum while active.
len(df_swhacwa.user_id.unique())

183

In [13]:
df_swhacwa.groupby('user_id').path.count().mean()

163.02185792349727

In [14]:
len(df_swhacwa[df_swhacwa.program_name == 'data_science'].user_id.unique())

21

In [15]:
len(df_swhacwa[df_swhacwa.program_name == 'web_dev'].user_id.unique())

163

In [16]:
df_swhacwa.path.str[:8].value_counts()

javascri    7861
html-css    5976
/           1813
fundamen    1689
spring/f    1398
            ... 
arrays         1
mysql//e       1
advanced       1
11-nlp/2       1
sql.mysq       1
Name: path, Length: 177, dtype: int64

#### Students who hardly accessed the curriculum while active info
- 183 students accessed the curriculum < first quartile (391 times)
- 163 of these students are from web_dev programs
- 21 are from data science programs
- 1 student is in both programs
- Average access count of these students is 163 times
- ~ 30k total access count

### Q6: What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [17]:
# Post-grad access
df_post = df[df.date > df.end_date]

In [18]:
# Find data science top paths
df_post[df_post.program_name == 'data_science'].path.str[:10].value_counts().head(10)

/             1436
fundamenta    1052
classifica    1034
search/sea     493
1-fundamen     462
anomaly-de     452
6-regressi     375
timeseries     340
regression     335
sql/mysql-     324
Name: path, dtype: int64

In [19]:
# Find web dev top paths
df_post[df_post.program_name == 'web_dev'].path.str[:10].value_counts().head(20)

javascript    27284
/             14088
spring/fun    10260
html-css/c     7524
spring         4261
search/sea     3681
html-css       3678
java-iii       3537
java-ii        3439
content/ph     3436
java-i         3123
appendix       3071
classifica     2833
jquery/aja     2689
mysql          2574
toc            2564
jquery         2464
fundamenta     2358
java-iii/j     2271
java-iii/s     2052
Name: path, dtype: int64

#### Data Science Graduates Top Access
- Fundamentals
- Classification
- Regression
- Anomaly Detection
- Time Series

#### Web Dev Graduates Top Access
- Javascript
- HTML-CSS
- MySQL
- Fundamentals

### Q4: Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [20]:
df = df.set_index(df.date)

In [21]:
def prep(df, user):
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages = df['path'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plt_bands(my_df, user):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

def find_anomalies(df, user, span, weight):
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    # plt_bands(my_df, user)
    return my_df[my_df.pct_b>1]

In [22]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [23]:
anomalies.pages.value_counts(sort=False)

1      42
2      35
3      65
4      74
5      52
       ..
165     1
179     1
192     1
198     1
272     1
Name: pages, Length: 97, dtype: int64

In [24]:
sus_users = anomalies[anomalies.pages > 30].user_id.to_list()

In [25]:
df['is_sus'] = df['user_id'].isin(sus_users)

In [29]:
ip_address = df[df.is_sus == True].ip.to_list()

In [39]:
ip_address = np.unique(ip_address)

In [41]:
len(ip_address)

1290

In [44]:
import json
import requests
import time

In [45]:
ip_result = {}
for ip in ip_address:
    request_url = 'https://ipgeolocation.abstractapi.com/v1/?api_key=' + geo_key + '&ip_address=' + ip
    response = requests.get(request_url)
    result = json.loads(response.content)
    ip_result[ip] = result
    time.sleep(1.01)



In [73]:
for ip in ip_result:


TypeError: string indices must be integers

In [96]:
countries = []
ip_list = []
for ip in ip_result:
    countries.append(ip_result[ip]['country'])
    ip_list.append(ip)

In [98]:
countries = pd.DataFrame(countries)
ip_list = pd.DataFrame(ip_list)

In [106]:
sus_ip_countries = pd.merge(countries,ip_list,how='outer',left_index=True,right_index=True)

In [109]:
sus_ip_countries.rename(columns={'0_x':'country','0_y':'ip'},inplace = True)

In [112]:
sus_ip_countries.to_csv('suspicious_ip_countries.csv')

In [113]:
sus_ip_countries.country.value_counts()

United States    1281
Mexico              4
Germany             3
Switzerland         1
Canada              1
Name: country, dtype: int64

Odd access from Mexico, Germany, Switzerland, Canada